In [1]:
import requests
import pandas as pd
from pandas import json_normalize
import json
import random
from datetime import datetime, timedelta


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\jevan\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\jevan\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\jevan\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\jevan\AppData\Roaming\Python\Python310\site-packages\traitlets\config\applicatio

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\jevan\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\jevan\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\jevan\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\jevan\AppData\Roaming\Python\Python310\site-packages\traitlets\config\applicatio

AttributeError: _ARRAY_API not found

In [2]:
## Tentukan range waktu untuk generate timestamp
start = datetime(2025, 10, 1)
end = datetime(2025, 10, 31, 23, 59, 59)

In [18]:
## sementara dari local
r_product = requests.get('https://dummyjson.com/products?limit=200')
r_carts = requests.get('https://dummyjson.com/carts?limit=50')

## Load data products
df_product = json_normalize(r_product.json()['products'])

## Load data carts
df_sales = pd.DataFrame()
for item in r_carts.json()['carts']:
    random.seed(int(item['id']))
    df_temp = json_normalize(item['products'])
    df_temp['transaction_date'] = start + (end - start) * random.random()
    df_temp['cartID'] = item['id']
    df_sales = pd.concat([df_sales,df_temp],ignore_index=True)

In [19]:
df_product = df_product[['id','title','description','category','price']]
df_sales = df_sales[['id','title','price','quantity','transaction_date','cartID']]

In [20]:
df_product['category_id'] = df_product['category'].astype('category').cat.codes + 1

In [32]:
df_merge = pd.merge(df_product,df_sales,on='id',how='inner')
df_merge = df_merge[['id','title_x','category','quantity','price_y','transaction_date','cartID','category_id']].rename(columns ={
        'id': 'product_id',
        'title_x': 'product_name',
        'price_y': 'price', 
    }
)
df_merge['total_sales'] = df_merge['quantity']*df_merge['price']
df_merge.sort_values(by=['transaction_date','category_id'], ascending=[True,True], inplace=True)
df_merge.insert(0,'transaction_id', range(1,len(df_merge)+1))

In [39]:
df_final = df_merge[['transaction_id','product_id','product_name','category_id','quantity','price','total_sales','transaction_date']].rename(
    columns={'category_id':'category'})
df_final.set_index('transaction_id',inplace=True)

In [55]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("Ecommerce.db")

conn.execute("""
CREATE TABLE IF NOT EXISTS sales (
    transaction_id INTEGER PRIMARY KEY,
    product_id INTEGER NOT NULL,
    product_name TEXT NOT NULL,
    category INTEGER,
    quantity INTEGER CHECK(quantity > 0), 
    price REAL NOT NULL CHECK(price > 0),
    total_sales REAL,
    transaction_date TEXT
)       
""")

conn.execute("DELETE FROM sales")

try:
    df_final.to_sql('sales',conn,if_exists='append')
except Exception as e:
    print("Error: ", e)

conn.close()


In [54]:
conn = sqlite3.connect("Ecommerce.db")

with open("sales.sql", "w", encoding="utf-8") as f:
    for line in conn.iterdump():
        f.write(f"{line}\n")

conn.close()